# Group 7 Team Project

## This is the Python notebook for Group 7's (Kian, Kush, David, Rudy, and Chris) first presentation on P2P lending investment.
The following will hold brainstorming and data viewing so that it is accessible.

### Import Libraries and mount drive

In [1]:
import numpy as np
import pandas as pd
import pickle
pd.options.display.max_rows = None
pd.options.display.max_columns = None

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Support functions for much later modeling
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# Classification Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Scoring Functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## Define functions that can be generalised to clean data

In [2]:
# Code created by Jack Snoeyink and Daniel Ringel

def get_outliers(data, columns):
    # we create an empty list
    outlier_idxs = []
    # Number of standard deviations we keep. 
    nsd = 3 
    for col in columns:
        elements = data[col]
        # we get the mean value for each column
        mean = elements.mean()
        # and the standard deviation of the column
        sd = elements.std()
        # we then get the index values of all values higher or lower than the mean +/- nsd standard deviations
        outliers_mask = data[(data[col] > mean + nsd*sd) | (data[col]  < mean  - nsd*sd)].index
        # and add those index values to our list
        outlier_idxs  += [x for x in outliers_mask]
    return list(set(outlier_idxs))

Brainstorming and Data wrangling/cleaning

In [3]:
# the following two lines read in the accepted and declined data from csv files and parses them to pandas dataframes.
# the LCDataDictionary holds the definitions of some categories
# accepted_loand = pd.read_csv("accepted_2007_to_2018Q4.csv")
# rejected_loans = pd.read_csv("rejected_2007_to_2018Q4.csv")
# print(f"\n Number of Rows and Columns in Accepted Loans: {accepted_loans.shape}\n")
# print(f"\n Number of Rows and Columns in Rejected Loans: {rejected_loans.shape}\n")

# Declare necessary columns to minimize RAM usage
necessary_accepted = ['acc_open_past_24mths', 'all_util', 'annual_inc', 'application_type', 'avg_cur_bal', 'bc_util', 'grade', 
                      'home_ownership', 'il_util', 'installment', 'int_rate', 'loan_amnt', 'loan_status', 'mort_acc', 'num_op_rev_tl', 'num_rev_accts',
                      'num_rev_tl_bal_gt_0', 'num_sats', 'pct_tl_nvr_dlq', 'pymnt_plan', 'sub_grade', 'term', 'tot_cur_bal', 'tot_hi_cred_lim',
                      'verification_status',  'sec_app_fico_range_low', 'orig_projected_additional_accrued_interest',
                      'hardship_payoff_balance_amount']
necessary_rejected = ['acc_now_delinq','chargeoff_within_12_mths','collections_12_mths_ex_med','delinq_2yrs','delinq_amnt','inq_fi','inq_last_12m',
                      'inq_last_6mths','mths_since_last_delinq','mths_since_last_major_derog','mths_since_rcnt_il','mths_since_recent_bc_dlq',
                      'mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m',
                      'percent_bc_gt_75','pub_rec_bankruptcies','tot_coll_amt','sec_app_fico_range_high']

# Read CSV into DataFrame
accepted_loans = pd.read_csv('accepted_2007_to_2018Q4.csv', usecols = necessary_accepted)
rejected_loans = pd.read_csv('accepted_2007_to_2018Q4.csv', usecols = necessary_rejected)

In [0]:
# subsetting the dataframes to remove the rows that are include non-completed loans
reference_loans = accepted_loans[accepted_loans['loan_status'] != ['Current', 'Charged Off', 'In Grace Period']]
reference_loans.head(10)

In [4]:
# printing the heads of the dataframes above
# showing head of data to test that it parsed properly

print(accepted_loans.head(10))

# print(rejected_loans.head(10))

   loan_amnt        term  int_rate  installment grade sub_grade  \
0     3600.0   36 months     13.99       123.03     C        C4   
1    24700.0   36 months     11.99       820.28     C        C1   
2    20000.0   60 months     10.78       432.66     B        B4   
3    35000.0   60 months     14.85       829.90     C        C5   
4    10400.0   60 months     22.45       289.91     F        F1   
5    11950.0   36 months     13.44       405.18     C        C3   
6    20000.0   36 months      9.17       637.58     B        B2   
7    20000.0   36 months      8.49       631.26     B        B1   
8    10000.0   36 months      6.49       306.45     A        A2   
9     8000.0   36 months     11.48       263.74     B        B5   

  home_ownership  annual_inc verification_status loan_status pymnt_plan  \
0       MORTGAGE     55000.0        Not Verified  Fully Paid          n   
1       MORTGAGE     65000.0        Not Verified  Fully Paid          n   
2       MORTGAGE     63000.0        N

# Type casting data types to display appropriately in DataFrame

In [5]:
accepted_loans.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 28 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   loan_amnt                                   float64
 1   term                                        object 
 2   int_rate                                    float64
 3   installment                                 float64
 4   grade                                       object 
 5   sub_grade                                   object 
 6   home_ownership                              object 
 7   annual_inc                                  float64
 8   verification_status                         object 
 9   loan_status                                 object 
 10  pymnt_plan                                  object 
 11  application_type                            object 
 12  tot_cur_bal                                 float64
 13  il_util                    

In [6]:
rejected_loans.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 22 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   delinq_2yrs                     float64
 1   inq_last_6mths                  float64
 2   mths_since_last_delinq          float64
 3   collections_12_mths_ex_med      float64
 4   mths_since_last_major_derog     float64
 5   acc_now_delinq                  float64
 6   tot_coll_amt                    float64
 7   mths_since_rcnt_il              float64
 8   inq_fi                          float64
 9   inq_last_12m                    float64
 10  chargeoff_within_12_mths        float64
 11  delinq_amnt                     float64
 12  mths_since_recent_bc_dlq        float64
 13  mths_since_recent_revol_delinq  float64
 14  num_accts_ever_120_pd           float64
 15  num_tl_120dpd_2m                float64
 16  num_tl_30dpd                    float64
 17  num_tl_90g_dpd_24m         

In [7]:
accepted_loans.describe()

,loan_amnt,int_rate,installment,annual_inc,tot_cur_bal,il_util,all_util,acc_open_past_24mths,avg_cur_bal,bc_util,mort_acc,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,tot_hi_cred_lim,sec_app_fico_range_low,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount
count,2.260668e+06,2.260668e+06,2.260668e+06,2.260664e+06,2.190392e+06,1.191818e+06,1.394320e+06,2.210638e+06,2.190322e+06,2.184597e+06,2.210638e+06,2.190392e+06,2.190391e+06,2.190392e+06,2.202078e+06,2.190237e+06,2.190392e+06,108021.000000,8651.000000,10917.000000
mean,1.504693e+04,1.309283e+01,4.458068e+02,7.799243e+04,1.424922e+05,6.914098e+01,5.703230e+01,4.521656e+00,1.354780e+04,5.789995e+01,1.555382e+00,8.246523e+00,1.400463e+01,5.577951e+00,1.162813e+01,9.411458e+01,1.782428e+05,669.755603,454.798089,11636.883942
std,9.190245e+03,4.832138e+00,2.671735e+02,1.126962e+05,1.606926e+05,2.374839e+01,2.090475e+01,3.164229e+00,1.647408e+04,2.858347e+01,1.904981e+00,4.683928e+00,8.038868e+00,3.293434e+00,5.644027e+00,9.036140e+00,1.815748e+05,44.729163,375.385500,7625.988281
min,5.000000e+02,5.310000e+00,4.930000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,540.000000,1.920000,55.730000
25%,8.000000e+03,9.490000e+00,2.516500e+02,4.600000e+04,2.909200e+04,5.500000e+01,4.300000e+01,2.000000e+00,3.080000e+03,3.540000e+01,0.000000e+00,5.000000e+00,8.000000e+00,3.000000e+00,8.000000e+00,9.130000e+01,5.073100e+04,645.000000,175.230000,5627.000000
50%,1.290000e+04,1.262000e+01,3.779900e+02,6.500000e+04,7.924000e+04,7.200000e+01,5.800000e+01,4.000000e+00,7.335000e+03,6.020000e+01,1.000000e+00,7.000000e+00,1.200000e+01,5.000000e+00,1.100000e+01,1.000000e+02,1.142985e+05,670.000000,352.770000,10028.390000
75%,2.000000e+04,1.599000e+01,5.933200e+02,9.300000e+04,2.132040e+05,8.600000e+01,7.200000e+01,6.000000e+00,1.878300e+04,8.310000e+01,3.000000e+00,1.000000e+01,1.800000e+01,7.000000e+00,1.400000e+01,1.000000e+02,2.577550e+05,695.000000,620.175000,16151.890000
max,4.000000e+04,3.099000e+01,1.719830e+03,1.100000e+08,9.971659e+06,1.000000e+03,2.390000e+02,6.400000e+01,9.580840e+05,3.396000e+02,9.400000e+01,9.100000e+01,1.510000e+02,6.500000e+01,1.010000e+02,1.000000e+02,9.999999e+06,845.000000,2680.890000,40306.410000


Get information about the data stored in the new dataframes

In [0]:
# Check data types of variables
accepted_loans.dtypes

In [9]:
# See the data
accepted_loans.head(10)

,loan_amnt,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,application_type,tot_cur_bal,il_util,all_util,acc_open_past_24mths,avg_cur_bal,bc_util,mort_acc,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,tot_hi_cred_lim,sec_app_fico_range_low,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount
0,3600.0,36 months,13.99,123.03,C,C4,MORTGAGE,55000.0,Not Verified,Fully Paid,n,Individual,144904.0,36.0,34.0,4.0,20701.0,37.2,1.0,4.0,9.0,4.0,7.0,76.9,178050.0,NaN,NaN,NaN
1,24700.0,36 months,11.99,820.28,C,C1,MORTGAGE,65000.0,Not Verified,Fully Paid,n,Individual,204396.0,73.0,29.0,4.0,9733.0,27.1,4.0,20.0,27.0,5.0,22.0,97.4,314017.0,NaN,NaN,NaN
2,20000.0,60 months,10.78,432.66,B,B4,MORTGAGE,63000.0,Not Verified,Fully Paid,n,Joint App,189699.0,73.0,65.0,6.0,31617.0,55.9,5.0,4.0,7.0,3.0,6.0,100.0,218418.0,NaN,NaN,NaN
3,35000.0,60 months,14.85,829.90,C,C5,MORTGAGE,110000.0,Source Verified,Current,n,Individual,301500.0,70.0,45.0,2.0,23192.0,12.1,1.0,10.0,13.0,5.0,13.0,100.0,381215.0,NaN,NaN,NaN
4,10400.0,60 months,22.45,289.91,F,F1,MORTGAGE,104433.0,Source Verified,Fully Paid,n,Individual,331730.0,84.0,78.0,10.0,27644.0,77.5,6.0,7.0,19.0,6.0,12.0,96.6,439570.0,NaN,NaN,NaN
5,11950.0,36 months,13.44,405.18,C,C3,RENT,34000.0,Source Verified,Fully Paid,n,Individual,12798.0,99.0,76.0,0.0,2560.0,91.0,0.0,4.0,4.0,3.0,5.0,100.0,16900.0,NaN,NaN,NaN
6,20000.0,36 months,9.17,637.58,B,B2,MORTGAGE,180000.0,Not Verified,Fully Paid,n,Individual,360358.0,63.0,74.0,6.0,30030.0,102.9,4.0,9.0,16.0,6.0,12.0,96.3,388852.0,NaN,NaN,NaN
7,20000.0,36 months,8.49,631.26,B,B1,MORTGAGE,85000.0,Not Verified,Fully Paid,n,Individual,141601.0,75.0,55.0,4.0,17700.0,5.7,3.0,3.0,3.0,2.0,8.0,93.3,193390.0,NaN,NaN,NaN
8,10000.0,36 months,6.49,306.45,A,A2,RENT,85000.0,Not Verified,Fully Paid,n,Individual,27957.0,57.0,46.0,7.0,1997.0,50.1,1.0,13.0,19.0,9.0,14.0,95.7,61099.0,NaN,NaN,NaN
9,8000.0,36 months,11.48,263.74,B,B5,MORTGAGE,42000.0,Not Verified,Fully Paid,n,Individual,199696.0,72.0,49.0,5.0,28528.0,41.4,1.0,5.0,11.0,3.0,8.0,94.4,256513.0,NaN,NaN,NaN


In [10]:
# Count number of unique values of variables
accepted_loans.nunique()

In [11]:
# Count number of null values
print(accepted_loans.isnull().sum())

loan_amnt                                          33
term                                               33
int_rate                                           33
installment                                        33
grade                                              33
sub_grade                                          33
home_ownership                                     33
annual_inc                                         37
verification_status                                33
loan_status                                        33
pymnt_plan                                         33
application_type                                   33
tot_cur_bal                                     70309
il_util                                       1068883
all_util                                       866381
acc_open_past_24mths                            50063
avg_cur_bal                                     70379
bc_util                                         76104
mort_acc                    

In [0]:
accepted_loans.dropna(axis=1, thresh=1000000)

## Data Validity
Begin changing data into its proper types

# Impute missing data

In [12]:
import missingno as msno
%matplotlib inline